# Getting Data


## Getting data from database

In [32]:
import sqlite3 as lite
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import statsmodels.formula.api as sm
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression # this is the most important
# according to her lecture
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [33]:
#Functions used in the code
def GetAllWeekNos(start, end):
    c.execute("SELECT DISTINCT week_no from timetable where week_no >= '" \
              + start+ "' AND week_no <= '"+ end+ "'")
    data = c.fetchall()
    week_nos = []
    for weeks in data:
        #print(weeks)
        list_item = weeks[0].split("/")
        list_item = [ int(x) for x in list_item ]
        list_item.append(weeks[0])
        week_nos.append(list_item)
        #print(week_nos)
    return week_nos

def WeekNo(date):
    weekno = date.isocalendar()
    return weekno

def FormatWeekNo(week):
    return str(week[1]) + "/" + str(week[0])

def GetWeek(date):
    ''' get the relevant week from the timetable table'''
    
    week = date.isocalendar()
    week_nos = FormatWeekNo(week)
    
    if week[1] >= WeekNo(parse("1 Sept" + str(week[0])))[1]:
        semester_beg = WeekNo(parse("1 Sept" + str(week[0])))
        semester_end = WeekNo(parse("31 Dec" + str(week[0]))) 
    else:
        semester_beg = WeekNo(parse("1 Jan" + str(week[0])))
        semester_end = WeekNo(parse("30 May" + str(week[0])))
    
    semester_beg = FormatWeekNo(semester_beg)
    semester_end = FormatWeekNo(semester_end)
    week_nos = GetAllWeekNos(semester_beg, semester_end)
    
    if week_nos[0][1] % 2:
        even = week_nos[0][2]
        odd = week_nos[1][2]
    else:
        even = week_nos[1][2]
        odd = week_nos[0][2]
    
    if week[1] % 2:
        return even
    else:
        return odd



In [34]:
con = lite.connect('wicount.sqlite3')
c=con.cursor()
with con:    
    c = con.cursor()    
    c.execute("SELECT * FROM survey")

    rows = c.fetchall()

    for row in rows:
        print(row)

(3, '2015-11-02 09:00:00', 'Mon', 0.25)
(1, '2015-11-02 09:00:00', 'Mon', 0.25)
(2, '2015-11-02 09:00:00', 'Mon', 0.25)
(4, '2015-11-02 09:00:00', 'Mon', 1.0)
(5, '2015-11-02 09:00:00', 'Mon', 0.0)
(6, '2015-11-02 09:00:00', 'Mon', 0.0)
(3, '2015-11-02 10:00:00', 'Mon', 0.5)
(1, '2015-11-02 10:00:00', 'Mon', 0.5)
(2, '2015-11-02 10:00:00', 'Mon', 0.5)
(4, '2015-11-02 10:00:00', 'Mon', 1.0)
(5, '2015-11-02 10:00:00', 'Mon', 0.5)
(6, '2015-11-02 10:00:00', 'Mon', 0.5)
(3, '2015-11-02 11:00:00', 'Mon', 0.25)
(1, '2015-11-02 11:00:00', 'Mon', 0.25)
(2, '2015-11-02 11:00:00', 'Mon', 0.5)
(4, '2015-11-02 11:00:00', 'Mon', 0.25)
(5, '2015-11-02 11:00:00', 'Mon', 0.0)
(6, '2015-11-02 11:00:00', 'Mon', 0.5)
(3, '2015-11-02 12:00:00', 'Mon', 0.25)
(1, '2015-11-02 12:00:00', 'Mon', 0.25)
(2, '2015-11-02 12:00:00', 'Mon', 0.5)
(4, '2015-11-02 12:00:00', 'Mon', 0.25)
(5, '2015-11-02 12:00:00', 'Mon', 0.5)
(6, '2015-11-02 12:00:00', 'Mon', 0.5)
(3, '2015-11-02 13:00:00', 'Mon', 0.5)
(1, '2015-11-02 

In [35]:
full_data = []
data_list = []

for row in rows:
    c.execute("SELECT * FROM room WHERE room_id = '" + str(row[0]) + "'")
    college = c.fetchall()[0]
    
    #get time and date fields.
    fromDate = datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S")
    toDate = fromDate + timedelta(hours=1)
    time = datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S").time()
    week_no = GetWeek(fromDate)
    
    #only continue if we have timetable data.
    c.execute("SELECT * FROM timetable WHERE room_id = '" + str(row[0]) + \
                "' and day = '" + row[2] + "' AND  time = '" + str(time) + "' AND week_no = '" + week_no + "'")
    timetable = c.fetchall()
    if timetable == []:
        continue
    timetable = timetable[0]

    c.execute("SELECT count, date FROM logdata WHERE room_id = '" + str(row[0]) + \
                 "' and date BETWEEN '" + str(fromDate) + "' AND '" + str(toDate) + "'")
    logData = c.fetchall()
    # Only use if we have all the data for all the days
    if logData == []:
        continue
    
    c.execute("SELECT percentage FROM survey WHERE room_id = '" + str(row[0]) + \
                "' and date = '" + str(fromDate) + "'")
    survey = c.fetchall()[0]
    #print("row: ", row)
    #print("college: ", college)
    #print("timetable: ", timetable)
    #print("logData: ", logData)
    #print ("survey: ", survey)
    #print('Campus: ',college[1])
    #print('Building: ',college[2])
    #print('Room: ',college[3])
    #print('Occupancy: ',college[4])
    #print('Date: ',row[1])
    #print('Day: ',row[2], ' SurveyPercentage: ',survey[0],)
    #print('MaxCount: ',logData[0], ' Module: ',timetable[3], ' NoStudents: ',timetable[4])
    
    #Change here for the output
    for i in range(0, len(logData)):
        data_list = [college[1], college[2], college[3], college[4], \
                 row[1], row[2], survey[0], \
                 logData[i][0], logData[i][1], timetable[4], timetable[5], timetable[3]]
        full_data.append(data_list)
    
data = pd.DataFrame(full_data, columns=('Campus', 'Building', 'Room', 'Capacity', \
                          'Date', 'Day', 'SurveyPercentage', \
                            'Count', 'LogDate', 'Module', 'NoStudents', 'WeekNo'))
data.head(2)
print(data)
#output to a file
data.to_csv("full_dataset.csv")


        Campus          Building   Room  Capacity                 Date  Day  \
0     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
1     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
2     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
3     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
4     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
5     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
6     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
7     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
8     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
9     Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
10    Belfield  Computer Science  B-004       160  2015-11-03 09:00:00  Tue   
11    Belfield  Computer Science  B-004       160  2

In [36]:
# Show quick statistic summary of continuous features
print("Continuous Features")
data.describe().T

Continuous Features


,count,mean,std,min,25%,50%,75%,max
Capacity,2502,113.333333,33.004913,90,90,90.00,160.00,160
SurveyPercentage,2502,0.249101,0.217957,0,0,0.25,0.25,1
Count,2502,30.284972,28.965944,0,10,22.00,42.00,232


In [37]:
print("Feature, UniqueValues") 

#Look at the numerical features only
numeric_columns = data.select_dtypes(['int64', 'float64']).columns

for column in numeric_columns:
    print(column + ", \t" + str(len(data[column].unique())))

Feature, UniqueValues
Capacity, 	2
SurveyPercentage, 	5
Count, 	141


In [38]:
#Select all columns of type 'object'
object_columns = data.select_dtypes(['object']).columns
object_columns

#Convert selected columns to type 'category'
for column in object_columns:
    data[column] = data[column].astype('category')
data.dtypes   

Campus              category
Building            category
Room                category
Capacity               int64
Date                category
Day                 category
SurveyPercentage     float64
Count                  int64
LogDate             category
Module              category
NoStudents          category
WeekNo              category
dtype: object

In [39]:
#Look at the categorical features only
category_columns = data.select_dtypes(['category']).columns
data[category_columns]

# Descriptive stats for categorical features
data.select_dtypes(['category']).describe().T

,count,unique,top,freq
Campus,2502,1,Belfield,2502
Building,2502,1,Computer Science,2502
Room,2502,3,B-004,834
Date,2502,72,2015-11-13 16:00:00,36
Day,2502,5,Fri,564
LogDate,2502,834,2015-11-13 16:57:35,3
Module,1817,38,COMP30080,138
NoStudents,1817,32,0,279
WeekNo,2502,2,46/2015,1395


In [40]:
#data = data[np.isfinite(data['Count'])]
#data = data[np.isfinite(data['Capacity'])]
#data = data[np.isfinite(data['SurveyPercentage'])]
data.head(10)

,Campus,Building,Room,Capacity,Date,Day,SurveyPercentage,Count,LogDate,Module,NoStudents,WeekNo
0,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,62,2015-11-03 09:02:20,COMP41450 & COMP30120,143,45/2015
1,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,95,2015-11-03 09:07:24,COMP41450 & COMP30120,143,45/2015
2,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,113,2015-11-03 09:12:01,COMP41450 & COMP30120,143,45/2015
3,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,119,2015-11-03 09:16:53,COMP41450 & COMP30120,143,45/2015
4,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,124,2015-11-03 09:21:55,COMP41450 & COMP30120,143,45/2015
5,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,122,2015-11-03 09:27:00,COMP41450 & COMP30120,143,45/2015
6,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,124,2015-11-03 09:32:07,COMP41450 & COMP30120,143,45/2015
7,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,125,2015-11-03 09:37:18,COMP41450 & COMP30120,143,45/2015
8,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,125,2015-11-03 09:42:20,COMP41450 & COMP30120,143,45/2015
9,Belfield,Computer Science,B-004,160,2015-11-03 09:00:00,Tue,0.75,130,2015-11-03 09:47:30,COMP41450 & COMP30120,143,45/2015


In [106]:
percent = data.SurveyPercentage
bottomLow = []
bottomHigh = []
TopLow = []
TopHigh = []
for a in percent:
    #print(a)
    if a <= 0.25:
        #print("bottomLow")
        bottomLow.append(1)
        bottomHigh.append(0)
        TopLow.append(0)
        TopHigh.append(0)
    elif a <= 0.5:
        #print("bottomHigh")
        bottomLow.append(0)
        bottomHigh.append(1)
        TopLow.append(0)
        TopHigh.append(0)
    elif a <= 0.75:
        #print("TopLow")
        bottomLow.append(0)
        bottomHigh.append(0)
        TopLow.append(1)
        TopHigh.append(0)
    else:
        #print("TopHigh")
        bottomLow.append(0)
        bottomHigh.append(0)
        TopLow.append(0)
        TopHigh.append(1)
    
data['bottomLow'] = bottomLow
data['bottomHigh'] = bottomHigh
data['TopLow'] = TopLow
data['TopHigh'] = TopHigh
data.head(10)
data.to_csv("extra_dataset.csv")

In [109]:
logreg = sm.logit(formula="bottomLow ~   Count + Capacity", data=data).fit()
#logregBH = sm.logit(formula="bottomHigh ~   Count + Capacity", data=data).fit()
#logregTL = sm.logit(formula="TopLow ~   Count + Capacity", data=data).fit()
#logregTH = sm.logit(formula="TopHigh ~   Count + Capacity", data=data).fit()

# Print the weights learned for each feature.
print(logreg.params)

Optimization terminated successfully.
         Current function value: 0.455965
         Iterations 6
Intercept    0.956971
Count       -0.044928
Capacity     0.015437
dtype: float64


In [110]:
#From lecture: Scikit expects a X matrix with discriptive features.
# need to explicitly get the intercept
# Prepare an intercept term that adds a 1 to each example.
# Scikit-learn does not use an intercept by default.
intercept = pd.DataFrame({'Intercept': np.ones(data.shape[0])})
intercept

,Intercept
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [111]:
# Prepare the descriptive features
X = pd.concat([intercept, data[['Count', 'Capacity']]], axis=1)
y = data.bottomLow
print("Descriptive features:\n", X)
print("\nTarget feature:\n", y)

Descriptive features:
       Intercept  Count  Capacity
0             1     62       160
1             1     95       160
2             1    113       160
3             1    119       160
4             1    124       160
5             1    122       160
6             1    124       160
7             1    125       160
8             1    125       160
9             1    130       160
10            1     68       160
11            1     20       160
12            1      2        90
13            1      1        90
14            1      0        90
15            1      0        90
16            1      1        90
17            1      2        90
18            1      1        90
19            1      4        90
20            1      4        90
21            1      3        90
22            1     18        90
23            1     21        90
24            1      2        90
25            1      1        90
26            1      1        90
27            1      1        90
28            1     

In [112]:
# Train a model using logistic regression from scikit-learn.
# Use only the descriptive feature Size.
logreg = LogisticRegression().fit(X[['Intercept', 'Count']], y)

In [113]:
# Examine the estimated logistic regression coefficients.
print("Coeficients: \n", logreg.coef_)
#pd.DataFrame(zip(X[['Size']], np.transpose(logreg.coef_)))

Coeficients: 
 [[ 1.18178116 -0.03605053]]


In [114]:
# Predicted probabilities for each example. 
# The output is a pair for each example, 
# The first component is the probability of the negative class (class 0).
# The second component is the probability of the positive class (class 1).
logreg.predict_proba(X[['Intercept', 'Count']])

array([[ 0.46793666,  0.53206334],
       [ 0.74292702,  0.25707298],
       [ 0.84685539,  0.15314461],
       ..., 
       [ 0.13909771,  0.86090229],
       [ 0.13483658,  0.86516342],
       [ 0.12664481,  0.87335519]])

In [115]:
# Predict using the trained model, on in-sample data (same sample used for training and test)
predictions = logreg.predict(X[['Intercept', 'Count']])
print("Predictions: ", predictions)
data['PredictBotLow'] = predictions

Predictions:  [1 0 0 ..., 1 1 1]


In [116]:
# Check the accuracy on the training set. 
# The accuracy is the ratio of correct predictions to total examples
# In our case we have 10 examples and the predictions is correct on 9 of them.
# Total accuracy is then 9/10 = 0.9 or 90% accuracy
logreg.score(X[['Intercept', 'Count']], y)

0.77418065547561954

In [117]:
#logreg = sm.logit(formula="bottomLow ~   Count + Capacity", data=data).fit()
logreg = sm.logit(formula="bottomHigh ~   Count + Capacity", data=data).fit()
#logregTL = sm.logit(formula="TopLow ~   Count + Capacity", data=data).fit()
#logregTH = sm.logit(formula="TopHigh ~   Count + Capacity", data=data).fit()

# Print the weights learned for each feature.
print(logreg.params)
intercept = pd.DataFrame({'Intercept': np.ones(data.shape[0])})
intercept
X = pd.concat([intercept, data[['Count', 'Capacity']]], axis=1)
y = data.bottomHigh
#print("Descriptive features:\n", X)
#print("\nTarget feature:\n", y)
logreg = LogisticRegression().fit(X[['Intercept', 'Count']], y)
#print("Coeficients: \n", logreg.coef_)
logreg.predict_proba(X[['Intercept', 'Count']])
predictions = logreg.predict(X[['Intercept', 'Count']])
data['PredictBotHigh'] = predictions
logreg.score(X[['Intercept', 'Count']], y)

Optimization terminated successfully.
         Current function value: 0.450756
         Iterations 6
Intercept   -1.989599
Count        0.020155
Capacity    -0.001657
dtype: float64


0.80415667466027174

In [118]:
#logreg = sm.logit(formula="bottomLow ~   Count + Capacity", data=data).fit()
#logreg = sm.logit(formula="bottomHigh ~   Count + Capacity", data=data).fit()
logreg = sm.logit(formula="TopLow ~   Count + Capacity", data=data).fit()
#logreg = sm.logit(formula="TopHigh ~   Count + Capacity", data=data).fit()

# Print the weights learned for each feature.
print(logreg.params)
intercept = pd.DataFrame({'Intercept': np.ones(data.shape[0])})
intercept
X = pd.concat([intercept, data[['Count', 'Capacity']]], axis=1)
y = data.TopLow
#print("Descriptive features:\n", X)
#print("\nTarget feature:\n", y)
logreg = LogisticRegression().fit(X[['Intercept', 'Count']], y)
#print("Coeficients: \n", logreg.coef_)
logreg.predict_proba(X[['Intercept', 'Count']])
predictions = logreg.predict(X[['Intercept', 'Count']])
data['PredictTopLow'] = predictions
logreg.score(X[['Intercept', 'Count']], y)

Optimization terminated successfully.
         Current function value: 0.154963
         Iterations 10
Intercept    2.662862
Count        0.050619
Capacity    -0.075644
dtype: float64


0.94084732214228617

In [119]:
#logreg = sm.logit(formula="bottomLow ~   Count + Capacity", data=data).fit()
#logreg = sm.logit(formula="bottomHigh ~   Count + Capacity", data=data).fit()
#logreg = sm.logit(formula="TopLow ~   Count + Capacity", data=data).fit()
logreg = sm.logit(formula="TopHigh ~   Count + Capacity", data=data).fit()

# Print the weights learned for each feature.
print(logreg.params)
intercept = pd.DataFrame({'Intercept': np.ones(data.shape[0])})
intercept
X = pd.concat([intercept, data[['Count', 'Capacity']]], axis=1)
y = data.TopHigh
#print("Descriptive features:\n", X)
#print("\nTarget feature:\n", y)
logreg = LogisticRegression().fit(X[['Intercept', 'Count']], y)
#print("Coeficients: \n", logreg.coef_)
logreg.predict_proba(X[['Intercept', 'Count']])
predictions = logreg.predict(X[['Intercept', 'Count']])
data['PredictTopHigh'] = predictions
logreg.score(X[['Intercept', 'Count']], y)

         Current function value: 0.007843
         Iterations: 35
Intercept   -39.922192
Count         0.050897
Capacity      0.190264
dtype: float64


C:\Users\veldi_000\Anaconda3\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.99920063948840931

In [120]:
data.to_csv("Predict_dataset.csv")